# README

Welcome to Jupyter!

This notebook is designed for demonstrating the capabilities of the [Spatial DS Stack](https://github.com/zacharlie/spatial-ds-stack), which is a simple stack for connecting a bespoke data science environment for exploratory data analysis (EDA).

The services made available in the stack include:

- Jupyter: The interactive noteboook and coding environment
- Python libraries: Such as pandas, seaborn, dask, pycaret, and scikit-learn
- PostgreSQL: A local database service
- PgAdmin: Database administration frontend 
- MLflow: A machine learning lifecycle management system

A flatfile=>geodata ingestion service is provided alongside the stack to push data from the filesystem into the database.

> Make sure that you have completed the instructions in the README.md file included with this repository to ensure that your notebook environment is properly configured before attempting the following steps..

## Database Connection Configuration

To interact with the database, we will have to configure connections. Configuration of this step is important due to the security implications of exposing db connection details.

One way or another, if you can load the config into a workbook then those config values are probably accessible to anyone using that workbook, most likely in an unencrypted manner.

The stack uses the configparser from the python standard library and loading of the config file as a docker secret.

In [ ]:
from configparser import ConfigParser

ConfigParser is a python standard module for loading configuration settings from INI formatted sources. By default, your database and environment configuration should include the `/nb.cfg` configuration file which is loaded as a docker secret. 

> Note that because it is loaded into the Jupyter container, anyone with access to this environment can extract the data from that file!

In [ ]:
# test configuration setup

CONFIG = ConfigParser()
CONFIG.read('/nb.cfg')
print(f"Your configured database user is {CONFIG['db']['USERNAME']}")

In [ ]:
# set environment variables from config
LOCAL_POSTGRES_HOST = CONFIG['db']['HOST']
LOCAL_POSTGRES_PORT = CONFIG['db']['PORT']
LOCAL_POSTGRES_DBNAME = CONFIG['db']['DBNAME']
LOCAL_POSTGRES_USERNAME = CONFIG['db']['USERNAME']
LOCAL_POSTGRES_PASSWORD = CONFIG['db']['PASSWORD']

In [ ]:
# define local connection string for sqlalchemy orm
import urllib

pg_conn_string_local = ('postgresql://{user}:{pword}@{host}:{port}/{dbname}'
                        .format(user=urllib.parse.quote_plus(LOCAL_POSTGRES_USERNAME),
                                pword=urllib.parse.quote_plus(LOCAL_POSTGRES_PASSWORD),
                                host=LOCAL_POSTGRES_HOST,
                                port=LOCAL_POSTGRES_PORT,
                                dbname=LOCAL_POSTGRES_DBNAME))

## Local DB Access with SQLAlchemy

SQLAlchemy is a python object relational mapper (ORM) for enabling interactions with SQL databases. We can use this to directly interact with supported databases, such as postgresql, with a set of classes which provide helper methods, input sanitization, and utility classes for database interaction. ORMs are most useful for developers unfamiliar with complex SQL operations, as well as simplifying or abstracting code for developer teams effectively whilst enforcing best practices.

In [ ]:
import sqlalchemy as db

In [ ]:
# Create the local connection
local_engine = db.create_engine(pg_conn_string_local)

In [ ]:
# test the local connection
local_inspector = db.inspect(local_engine)
local_inspector.get_schema_names()

In [ ]:
# view local tables and columns
for table_name in local_inspector.get_table_names():
    for column in local_inspector.get_columns(table_name):
        print(f"{table_name}.{column['name']}")

In [ ]:
# view local tables in the geodata schema
for table_name in local_inspector.get_table_names(schema='geodata'):
    print(f"{table_name}")

In [ ]:
with local_engine.connect() as connection:
    query_result = connection.execute(db.text('''
SELECT
    column_name,
    data_type
FROM
    information_schema.columns
WHERE
    table_schema = \'geodata\'
AND
    table_name = \'spatial_ref_sys\';
'''))

query_result.fetchall()

## Direct Access with psycopg2

A complex system such as the Data Gateway may not support connecting via an ORM like SQLAlchemy, but we can connect directly to the Data Integration HUB via a postgresql compatible. In this stack, we can use the included `psycopg2` library for this purpose.

In [ ]:
# define remote connection for data gateway with psycopg2
import psycopg2

connection = psycopg2.connect(
    host = REMOTE_POSTGRES_HOST,
    port = REMOTE_POSTGRES_PORT,
    dbname = REMOTE_POSTGRES_DBNAME,
    user = REMOTE_POSTGRES_USERNAME,
    password = REMOTE_POSTGRES_PASSWORD
)

In [ ]:
# Define a connection cursor
# (basically an instance of a database control structure)
cursor = connection.cursor()

In [ ]:
cursor.execute('''
  , col1
  , col2
FROM
  table1
LIMIT
  10
''')
result = cursor.fetchall()
# close the connection when completed
connection.close()
# Note that the retrieved result is still available in memory
result

In [ ]:
import pandas as pd

# load the query data into a data frame
df = pd.DataFrame(result)
df.columns = ['col1', 'col2']
df

## Data Retrieval and Exploration

Now that we've seen some database connectivity in action, let's have a look at how to retrieve data from the database and analyze it's structure.

First we need to access the metadata object for the database to setup the SQLAlchemy context.

In [ ]:
# create Metadata Object
local_meta = db.MetaData()
local_meta.reflect(local_engine)

The stack includes the `postgis` extension in the local database which creates a `spatial_ref_sys` table. We can query the table and retrieve records from the local database using python.

In [ ]:
# Get the Spatial Reference System table from the Metadata object
SRS = local_meta.tables['spatial_ref_sys']

# SQLAlchemy Query to select all rows based on some condition
query = db.select(SRS).where(SRS.c.srid > 10000)

# Fetch the records
with local_engine.connect() as cnx:
    results = cnx.execute(query)

i = 1

for result in results:
    if i <= 3:
        i += 1
        print(result)

Pandas is also capable of reading a SQL table directly into a dataframe. 

In [ ]:
import pandas as pd
with local_engine.connect() as cnx:
    df = pd.read_sql_table('spatial_ref_sys', cnx)

In [ ]:
# print the table
print(df.head(10))

In [ ]:
# display the table
df

In [ ]:
# row and column count
df.shape

In [ ]:
# column names
df.columns

In [ ]:
# show stats on numeric columns
df.describe()

In [ ]:
# show stats on df columns, types, and indexes
df.info()

In [ ]:
# let's clean up
del SRS, query, result, df

## AI  🐝º¤ø,¸¸,ø¤º°'°º¤ø,¸.

Artificial Intelligence, Machine Learning, Natural Language Processing, Large Language Models, and other facets of advanced business intelligence are exciting not because they're buzz words, but largely because very smart people have been fantasizing these technologies for a very long time and we are finally entering a stage in their lifecycle where they are capable of providing practical benefits, as well as becoming effective tools for developing similar tools to themselves increasing the rate of innovation substantially.

One of the keys to effectively leveraging these technological advancements is to be rational and pragmatic about their applications, strengths, and weaknesses.

It should be explicitly (and strongly) noted that most AI tools are not considered "production ready", have very serious (and largely unanswered) ethical implications, various privacy concerns, and most of all are generally experimental and prone to being glitchy and very often just plain wrong. They are also usually trained on very large low quality datasets, so in practically all cases YMMV.

That said, AI tools have seen extraordinary growth in recent times and are driving a significant paradigm shift in the way we work with and interact with technology. One thing they are terrific for are for producing quick and cheap to produce exploratory outputs to simple business questions, which can uncover a lot of value and provide the anecdotal evidence needed to warrant an discovery exercise and more formal proposal.

One of the most effective ways of realizing these benefits is using the various tools available to provide a conversational model which is context aware and trained (or provided background context) on a dataset to be evaluated so that the model can be asked questions about your business.

We've collected a few of these in this stack to get an intelligent data science ready environment up that is simple to use and configure. 

### ⚠ Personal Privacy considerations with AI Tools

One key consideration when utilizing AI tools is the appropriate licensing and utilization of data on the platform, as well as the exposure of private or sensitive data to such tools.

Personal Identifiable Information (PII) is protected by law in many jurisdictions, and exposure of such information to AI platforms may constitute an infraction with significant repercussions, or even a TOS violation on the platform itself.

Although this is largely case specific, it is possible (for the most part) to mitigate this risk by taking into account the following considerations:

- Understanding how a *specific* tool or library handles this particular issue is the first step forward
- Note that AI platforms may process or store your data and requests, and use it for further training
- Retraining models can expose sensitive data that is not PII, such as internal policy IDs
- Utilizing the "schema" or data structure of a set of records is typically safe as it does not constitute PII
- Anonymized or random data is usually safe for use (although effective processing is non-trivial to achieve without leaks or preventing the reverse engineering of PII)
- PII encompasses a lot of other data besides SSN, Name, or Email

### PandasAI

A simple ai tool for pushing contextual information from a dataframe to a LLM with a focus on simplicity.

In [ ]:
from configparser import ConfigParser
import pandas as pd
import pandasai

CONFIG = ConfigParser()
CONFIG.read('/nb.cfg')
OPENAI_API_KEY = CONFIG['openai']['OPENAI_API_KEY']
del CONFIG

# Sample DataFrame
df = pd.DataFrame({
    "country": ["United States", "United Kingdom", "France", "Germany", "Italy", "Spain", "Canada", "Australia", "Japan", "China"],
    "gdp": [19294482071552, 2891615567872, 2411255037952, 3435817336832, 1745433788416, 1181205135360, 1607402389504, 1490967855104, 4380756541440, 14631844184064],
    "happiness_index": [6.94, 7.16, 6.66, 7.07, 6.38, 6.4, 7.23, 7.22, 5.87, 5.12]
})

df

In [ ]:
df.describe()

In [ ]:
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI

# llm = OpenAI(openai_api_key=OPENAI_API_KEY)
llm = OpenAI(api_token=OPENAI_API_KEY)
# pandasai, hupyter, and docker have some caching issues 
# https://github.com/gventuri/pandas-ai/issues/269
# pandas_ai = PandasAI(llm=llm, enforce_privacy=True, enable_cache=False, verbose=True, conversational=True)  # debug
pandas_ai = PandasAI(llm=llm, enforce_privacy=True, enable_cache=False, verbose=False, conversational=True)
response = pandas_ai(df, 'Which are the 5 happiest countries?')
print(response)

### TODO

In [ ]:
# langchain
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
df = ''
chat = ChatOpenAI(model_name="gpt-4", temperature=0.0)
agent = create_pandas_dataframe_agent(chat, df, verbose=True)
agent.run('prompt')

In [ ]:
# https://lamini-ai.github.io

from llama import LLMEngine, Type, Context

llm = LLMEngine(
    id="example_llm",
    config={"production.key": "<YOUR-KEY-HERE>"}
    )

class Test(Type):
    test_string: str = Context("just a test")

llm = LLMEngine(id="my_test")

test = Test(test_string="testing 123")
llm(test, output_type=Test)
